# Text to Text  Explanation: Abstractive Summarization Example

This notebook demonstrates use of generating model explanations for a text to text scenario on a pretrained transformer model. Below we demonstrate the process of generating explanations for a pretrained model distilbart on the Extreme Summarization (XSum) Dataset provided by hugging face (https://huggingface.co/sshleifer/distilbart-xsum-12-6). 

The first example only needs the model and tokenizer and we use the model decoder to generate log odds of the output tokens to be explained. In the second example, we demonstrate the use of how to generate expplanations for model in the form of an api/fucntion (input->text and output->text). In this case we need to approximate the log odds by using a text similarity model. The underlying explainer used to compute the shap values is the partition explainer.

In [1]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import nlp
import shap

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (pyOpenSSL 20.0.1 (c:\users\v-maxtell\miniconda3\envs\interpret_cpu\lib\site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyOpenSSL 20.0.1 (c:\users\v-maxtell\miniconda3\envs\interpret_cpu\lib\site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (pyOpenSSL 20.0.1 (c:\users\v-maxtell\miniconda3\envs\interpret_cpu\lib\site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepR

### Load model and tokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-xsum-12-6")
model =  AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-xsum-12-6")

### Load data

In [3]:
dataset = nlp.load_dataset('xsum',split='train')

Using custom data configuration default


In [7]:
print(dataset)

Dataset(features: {'document': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None)}, num_rows: 204017)


In [4]:
# slice inputs from dataset to run model inference on
s = dataset['document'][0:1]

### Create an explainer object

In [5]:
explainer = shap.Explainer(model,tokenizer)

In [6]:
print(s)

['The problem is affecting people using the older versions of the PlayStation 3, called the "Fat" model.The problem isn\'t affecting the newer PS3 Slim systems that have been on sale since September last year.Sony have also said they are aiming to have the problem fixed shortly but is advising some users to avoid using their console for the time being."We hope to resolve this problem within the next 24 hours," a statement reads. "In the meantime, if you have a model other than the new slim PS3, we advise that you do not use your PS3 system, as doing so may result in errors in some functionality, such as recording obtained trophies, and not being able to restore certain data."We believe we have identified that this problem is being caused by a bug in the clock functionality incorporated in the system."The PlayStation Network is used by millions of people around the world.It allows users to play their friends at games like Fifa over the internet and also do things like download software 

### Compute shap values

In [6]:
shap_values = explainer(s)

Partition explainer: 2it [00:20, 20.83s/it]               


### Visualize shap explanations

In [7]:
shap.plots.text(shap_values)

,"The problem is affecting people using the older versions of the PlayStation 3,","called the ""Fat"" model.",The problem isn't affecting the newer PS,3 Slim systems that have been on sale since September last year.,Sony have also said they are aiming to have the problem fixed shortly but is advising,"some users to avoid using their console for the time being.""We hope to resolve this problem within the next 24 hours,"" a statement reads.","""In the meantime,","if you have a model other than the new slim PS3,","we advise that you do not use your PS3 system,","as doing so may result in errors in some functionality, such as recording obtained trophies,","and not being able to restore certain data.""We believe we have identified that this","problem is being caused by a bug in the clock functionality incorporated in the system.""The PlayStation Network is used by millions of people around the world.",It allows users to play their friends at games like Fifa over the internet and also do things like download software or visit online stores.
Sony,1.026,0.069,0.526,0.43,1.786,1.413,0.053,0.637,0.42,0.115,0.412,1.495,0.459
has,0.387,0.084,0.239,0.403,0.169,0.404,0.16,0.475,0.31,0.306,0.181,0.211,0.475
said,0.332,-0.178,0.175,-0.054,0.571,0.573,0.247,0.253,-0.117,-0.148,0.527,0.568,-0.408
that,0.352,0.19,0.196,0.343,-0.325,-0.232,0.003,-0.035,0.16,0.234,0.258,0.292,0.01
a,0.345,0.131,0.166,-0.091,0.303,0.691,0.004,0.087,0.317,-0.161,0.346,1.093,-0.097
bug,1.163,-0.133,0.628,0.012,0.141,0.377,0.034,-0.343,0.211,0.712,1.87,3.104,-0.279
in,0.098,0.01,0.171,0.126,0.108,0.141,0.151,0.294,0.196,0.59,0.35,1.029,0.122
its,-0.344,-0.352,-0.204,-0.21,-0.202,-0.03,0.031,-0.165,-0.188,-0.038,0.175,0.767,0.09
PlayStation,1.446,-0.09,0.733,0.268,1.283,1.372,0.13,0.94,0.943,0.115,0.302,3.129,0.524
3,0.983,0.42,0.662,0.758,0.719,0.24,0.115,1.176,1.764,0.435,-0.21,-1.162,-0.565


### API

Below we demonstrate generating explanations for a model which is an api/function. Since this is a model agnostic case, we use a text similarity model to approximate log odds of generating output text which is used for computing shap explanations.

In [8]:
# Define function
def f(x):
    input_ids = torch.tensor([tokenizer.encode(x)])
    with torch.no_grad():
        out=model.generate(input_ids)
    sentence = [tokenizer.decode(g, skip_special_tokens=True) for g in out][0]
    return sentence

For a model agnostic case, we wrap the model to be explained with the shal.models.TeacherForcingLogits class and define the text similarity model and tokenizer. The TeacherForcingLogits class uses the similarity model to approximate the log odds of generating the output text from the model(function->f)

We also have to create a Text masker and define mask_token="..." and pass collapse_mask_token=True, which then cues the algorithm to use text infilling while masking

In [9]:
# wrap model with TeacherForcingLogits class
wrapped_model = shap.models.TeacherForcingLogits(f, similarity_model=model, similarity_tokenizer=tokenizer)
# create a Text masker
masker = shap.maskers.Text(tokenizer, mask_token = "...", collapse_mask_token=True)

### Create an explainer object using wrapped model and Text masker

In [10]:
explainer_model_agnostic = shap.Explainer(wrapped_model,masker)

### Compute shap values

In [11]:
shap_values_model_agnostic = explainer_model_agnostic(s)

Partition explainer: 2it [01:43, 103.91s/it]              


### Visualize shap explanations

In [12]:
shap.plots.text(shap_values_model_agnostic)

,"The problem is affecting people using the older versions of the PlayStation 3,","called the ""Fat"" model.",The problem isn't affecting the newer PS,3 Slim systems that have been on sale since September last year.,Sony have also said they are aiming to,have the problem fixed shortly but is advising,"some users to avoid using their console for the time being.""We hope to resolve this problem within the next 24 hours,"" a statement reads.","""In the meantime, if you have a model other than the new slim PS3,","we advise that you do not use your PS3 system,","as doing so may result in errors in some functionality, such as recording obtained trophies,","and not being able to restore certain data.""We believe we have identified that this problem is being caused by a bug in the clock functionality incorporated in the system.""The PlayStation Network is used by millions of people around the world.",It allows users to play their friends at games like Fifa over the internet and also do things like download software or visit online stores.
Sony,1.016,0.434,0.488,0.256,1.017,0.144,-0.512,0.965,0.665,0.006,1.541,0.636
has,0.203,0.06,0.343,-0.036,0.306,0.169,0.262,0.297,0.334,0.036,0.672,0.173
said,0.391,-0.088,0.195,0.044,0.19,0.079,0.357,0.377,0.005,-0.062,0.603,-0.121
that,0.525,0.331,0.18,0.171,0.392,0.582,0.009,0.156,0.134,0.098,0.553,0.051
a,0.245,-0.01,0.147,0.016,0.011,0.08,-0.038,0.423,-0.002,-0.097,0.759,0.187
bug,0.893,0.025,0.517,-0.072,0.266,0.779,0.114,0.579,0.163,0.625,4.494,-0.025
in,0.449,0.095,0.034,0.056,0.772,0.438,0.249,0.37,-0.088,0.018,1.119,-0.045
its,-0.525,-0.175,-0.223,-0.133,-0.588,-0.763,-0.176,0.311,-0.072,0.095,0.592,-0.254
PlayStation,1.131,0.243,0.91,1.008,0.262,-0.171,0.426,1.03,0.969,0.544,2.538,-0.048
3,0.564,0.457,0.389,0.821,-0.216,-0.186,-0.149,0.568,0.924,0.568,1.583,-1.148
